Instructions for running notebook (on NCAR jupyterhub):  
1) Pull from github: `git clone https://github.com/josephko91/ice3d.git`
2) Make sure you are in the ice3d directory
3) Activate conda module: `module load conda`
4) Download required packages: `conda env create -f cq-conda-env.yaml -n cq`
5) Change the jupyter kernel to cq (or whatever you named it)

*NOTE: check file path for s_code.json

In [1]:
# import packages
import numpy as np
import cadquery as cq
import json
import random
import os

# (1) Load inputs

In [2]:
# Read the JSON file as a dictionary
# NOTE: when reading in from JSON, the key is read as a string
json_filepath = '/glade/u/home/joko/ice3d/s_code.json' # change directory as needed
with open(json_filepath, 'r') as json_file:
    s_code_dict = json.load(json_file)

# (2) Source Code

In [3]:
def create_bullet(a, c, hp, f_a, f_c, workplane):
    # create pyramid
    n_pyr = 6
    ri = a*np.cos(np.radians(30)) # distance between center and edge of hexagon
    theta = 90 - np.degrees(np.arctan(hp/ri))
    pyramid = workplane.polygon(n_pyr, f_a*2*a).extrude(-f_a*hp, taper=theta)
    # create cylinder 
    n_cyl = 6
    cylinder = workplane.polygon(n_cyl, f_a*2*a).extrude(f_c*2*c)
    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

def calc_r0(f_r0, a, n_arms):
    '''
    linearly interpolate between perscribed limits for r0
    '''
    ymin, ymax = 0.5*a, 1*a
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    r0 = slope*(n_arms) + intercept
    r0 = f_r0 * r0 # multiply by perturbation factor
    return r0 

def calc_hp(f_hp, r0, n_arms):
    '''
    linearly interpolate: hp increases with n_arms
    '''
    ymin, ymax = 1*r0, 1.5*r0
    xmin, xmax = 4, 12
    slope = (ymax-ymin)/(xmax-xmin)
    intercept = ymin - (slope*xmin)
    hp = slope*(n_arms) + intercept
    hp = f_hp*hp # multiply by perturbation factot
    return hp

def calc_h0(f_h0, r0):
    '''
    h0 calculate as a perscribed fraction of r0
    '''
    h0 = r0/2
    h0 = f_h0*h0 # multiply by perturbation factor
    return h0

def extract_xyz(s_code):
    '''
    Convert list in format [x1, y1, z1, ..., xn, yn, zn] to separate x, y, z arrays
    '''
    x = []
    y = []
    z = []
    for i in range(0, len(s_code), 3):
        x.append(s_code[i])
        y.append(s_code[i+1])
        z.append(s_code[i+2])
    return x, y, z

# create_ros(base_params, n_arms, s_code, aspect_perturb)
def create_ros(params, n_arms, s_code, aspect_perturb):
    '''
    aspect_perturb: list in form [f_a_1,f_c_1,...,f_a_n_arms,f_c_n_arms]
    '''
    # unpack parameters
    a, c, f_r0, f_hp, f_h0 = params[0], params[1], params[2], params[3], params[4]
    r0 = calc_r0(f_r0, a, n_arms)
    hp = calc_hp(f_hp, r0, n_arms)
    h0 = calc_h0(f_h0, r0)
    # create sphere
    sphere = cq.Workplane().sphere(r0)
    # create outer shell to "place" bullets on
    # based on spherical code from Sloane et al. 
    r_outer = r0 + hp - h0
    # convert s_code list to outer_coords
    x, y, z = extract_xyz(s_code)
    outer_coords = r_outer*(np.column_stack((x, y, z)))
    # create and collect bullets in list
    bullets = []
    for i in range(len(outer_coords)):
        f_a = aspect_perturb[2*i]
        f_c = aspect_perturb[2*i + 1]
        normal_vector = tuple(outer_coords[i])
        plane = cq.Plane(origin=normal_vector, normal=normal_vector)
        workplane = cq.Workplane(plane)
        bullet = create_bullet(a, c, hp, f_a, f_c, workplane)
        bullets.append(bullet)
    # boolean union to create rosette
    ros = sphere.union(bullets[0])
    for i in range(1, n_arms):
        ros = ros.union(bullets[i])
    return ros

# (3) Main Code

## (a) Load pre-generated paramater list

In [6]:
# read test subset of params
params_path = '/glade/u/home/joko/ice3d/output/params_200_50.json'
# load saved json
with open(params_path, 'rb') as file:
    params_list = json.load(file)

### (i) Check parameter distributions

In [27]:
print(params[0])
print(params[99])

[[26.53121070857551, 63.21278324652968, 1.0836825592696984, 0.9032904245911085, 0.8901685367597103, 4], [0.8763302560032733, 1.0531383801962657, 0.9479569137595074, 0.8395008445364098, 0.9434541951270421, 0.9217632083557027, 0.9565622867601465, 1.0925985207870723], [-0.26874508612934517, 0.9417069331058805, -0.20239597530990402, 0.2065136855284929, 0.8311269521861174, 0.5163139423250683, -0.13312816988630846, -0.708562720058195, 0.6929760184353092, 0.5795252100590425, -0.5252818587867077, -0.6230806526731543]]
[[26.53121070857551, 63.21278324652968, 1.0836825592696984, 0.9032904245911085, 0.8901685367597103, 4], [1.0501020528441458, 1.0469427269525389, 1.13524689256309, 0.9412838497771356, 1.1739504190529217, 1.1715651889368413, 1.0331251799429826, 0.8616827592199455], [-0.3561160498714929, 0.8459990373091887, -0.3968211031382405, 0.6946156820932747, 0.7192810990137289, -0.011989778634190554, -0.36593170478168796, -0.5794542937551687, 0.7282353389422035, 0.5217732776935582, -0.83032662

In [36]:
param_dists = {} # values of base parameters stored as dictionary
p_keys = ['a', 'c', 'f_r0','f_hp', 'f_h0']
# initialize dict
for j in range(len(p_keys)):
    param_dists[p_keys[j]] = []
for i in range(0, len(params), 100):
    p_set = params[i]
    vals = []
    for j in range(len(p_keys)):
        p = p_keys[j]
        v = p_set[0][j]
        param_dists[p].append(v)

In [37]:
len(param_dists['a'])

700

Note: look at notebook 08 for 2-d cross sections of the LHS.

## (b) Generate random rosette from parameter list

In [7]:
len(params_list)

70000

In [10]:
# Specify parameters
# params = random.choice(params_subset)
params = params_list[7]
base_params = params[0][:5]
n_arms = params[0][5]
aspect_perturb = params[1]
s_code = params[2]
print(f'base params: {base_params}')
print(f'n_arms: {n_arms}')
print(f'aspect_perturb: {aspect_perturb}')
print(f's_code: {s_code}')
# create rosette
ros = create_ros(base_params, n_arms, s_code, aspect_perturb)
print(f'rosette surface area: {ros.val().Area()}')
print(f'rosette volume: {ros.val().Volume()}')
ros

base params: [19.67777712915192, 26.434891792388346, 0.9227323103454409, 1.1209768452345905, 1.199872286962787]
n_arms: 4
aspect_perturb: [0.9751106278306524, 1.136250208478311, 1.1255783010533829, 0.8058509324538128, 1.0039472379097507, 1.167385068504915, 1.1342629729856328, 0.8449691985401457]
s_code: [-0.6237026248053689, 0.7711237569489464, -0.1279186744765414, 0.28361026993874683, 0.6885243569464349, 0.6674574328575339, -0.3120643354193718, -0.284831294419237, 0.9063591916446413, 0.4444461257634003, -0.7849031970841193, -0.43173442357661473]
rosette surface area: 32851.26545647104
rosette volume: 244341.86319514454


## (c) Save rosette as STL (mesh format)

In [ ]:
# if you want to save file
save_dir = '/glade/u/home/joko/ice3d/output'
save_filename = 'ros-test.stl'
save_filepath = os.path.join(save_dir, save_filename)
cq.exporters.export(ros, save_filepath)

# Scratch code

In [71]:
len(params_subset)

100

In [69]:
data = params_subset[:44]
num_tasks=10
for task_index in range(num_tasks):
    print(f'task index: {task_index}')
    # Calculate the chunk size for each task
    chunk_size = len(data) // num_tasks  # Integer division
    remainder = len(data) % num_tasks
    print(f'chunk size: {chunk_size}, remainder: {remainder}')
    # Calculate the start and end indices for the current task
    start_index = task_index * chunk_size
    if (task_index == (num_tasks-1)) & (remainder > 0):
        end_index = start_index + chunk_size + remainder 
    else:
        end_index = start_index + chunk_size 

    if start_index < len(data):
        print(start_index, end_index)
    else:
        pass

task index: 0
chunk size: 4, remainder: 4
0 4
task index: 1
chunk size: 4, remainder: 4
4 8
task index: 2
chunk size: 4, remainder: 4
8 12
task index: 3
chunk size: 4, remainder: 4
12 16
task index: 4
chunk size: 4, remainder: 4
16 20
task index: 5
chunk size: 4, remainder: 4
20 24
task index: 6
chunk size: 4, remainder: 4
24 28
task index: 7
chunk size: 4, remainder: 4
28 32
task index: 8
chunk size: 4, remainder: 4
32 36
task index: 9
chunk size: 4, remainder: 4
36 44


In [70]:
start_index, end_index = 0, 4
for i in range(start_index, end_index):
    print(i)

0
1
2
3
